In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from scipy.stats import entropy
import matplotlib.pyplot as plt
import seaborn as sns

# Read the data
pareto_models = pd.read_csv("pareto_hyperparameters.csv")
print(pareto_models.info())

# Assuming 'dt_simulated_weekly' and 'temp' data are also available in CSV files
dt_simulated_weekly = pd.read_csv("dt_simulated_weekly.csv")
temp = pd.read_csv("pareto_alldecomp_matrix.csv")


### Model selection

In [ ]:
# Sort the models based on a specific criterion
sorted_models = pareto_models.sort_values(by=['decomp.rssd'])

# Select the best and worst models
best_model = sorted_models.iloc[0]
worst_model = sorted_models.iloc[-1]

print(f"Best model solID: {best_model['solID']}")
print(f"Worst model solID: {worst_model['solID']}")

# Get the fits for these models
best_model_fits = temp[temp['solID'] == best_model['solID']]
worst_model_fits = temp[temp['solID'] == worst_model['solID']]


### KL Divergence Calculation

In [ ]:
# Function to calculate KL Divergence (adjust as needed)
def kl_divergence(p, q):
    return entropy(p, q)

# True values
true_vals = dt_simulated_weekly['revenue'][6:163]  # Adjust indices as necessary

# Calculate KL Divergence
kl_div_best = kl_divergence(true_vals, best_model_fits['depVarHat'])
kl_div_worst = kl_divergence(true_vals, worst_model_fits['depVarHat'])

print(f"KL Divergence for Best Model: {kl_div_best}")
print(f"KL Divergence for Worst Model: {kl_div_worst}")


### Visualization

In [ ]:
# Assuming you've calculated KL Divergence for a range of k values as needed
# Prepare the dataset for plotting

# Plotting code would depend on how the data for multiple k values is structured
# Here's a basic example of plotting with seaborn

sns.lineplot(data=your_kl_divergence_data, x="k", y="KL_divergence", hue="Model")
plt.title("KL Divergence of Best and Worst Model Fits")
plt.xlabel("Number of neighbours considered")
plt.ylabel("KL Divergence")
plt.show()


### Chebyshev's Inequality Analysis

In [ ]:
# Function to calculate the empirical probability
def empirical_prob(residuals, k):
    sigma = np.std(residuals)
    count = np.sum(np.abs(residuals) >= k * sigma)
    return count / len(residuals)

# Assuming residuals are calculated as the difference between predictions and true values
residuals_best = best_model_fits['depVarHat'] - true_vals
residuals_worst = worst_model_fits['depVarHat'] - true_vals

# Calculate empirical probabilities for different values of k
ks = [0.25, 0.5, 0.75, 1, np.sqrt(2), 1.5, 1.75, 2, 3, 4, 5]
probabilities_best = [empirical_prob(residuals_best, k) for k in ks]
probabilities_worst = [empirical_prob(residuals_worst, k) for k in ks]

# Theoretical bounds from Chebyshev's inequality
chebyshev_bounds = [min(1, 1 / k**2) for k in ks]

# Combine and compare the results
results_df = pd.DataFrame({
    'k': ks,
    'Best Model': probabilities_best,
    'Worst Model' : probabilities_worst})
